In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Olanrewaju Adegoke\\DLOPs\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\Olanrewaju Adegoke\\DLOPs\\Kidney_Disease_Classification'

In [6]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is F630-82A0

 Directory of c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification

07/31/2025  08:31 PM    <DIR>          .
07/31/2025  11:29 AM    <DIR>          ..
07/31/2025  12:13 PM    <DIR>          .github
07/31/2025  12:13 PM    <DIR>          config
07/31/2025  08:33 PM    <DIR>          data
07/31/2025  12:13 PM                 0 dvc.yaml
07/31/2025  02:11 PM    <DIR>          KIDNEY_CLASS_VENV
07/31/2025  03:34 PM    <DIR>          logs
07/31/2025  03:42 PM               127 main.py
07/31/2025  12:13 PM                 0 params.yaml
07/31/2025  11:24 PM             1,406 README.md
07/31/2025  02:42 PM               381 requirements.txt
07/31/2025  11:49 PM    <DIR>          research
07/31/2025  03:39 PM             1,042 setup.py
07/31/2025  12:43 PM    <DIR>          src
07/31/2025  12:12 PM             2,209 template.py
07/31/2025  12:13 PM    <DIR>          templates
               7 File(s)          5,165 byte

In [9]:
# Update the Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
#from src.Kidney_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

from src.Kidney_classifier.constants import *

from Kidney_classifier.utils.common import read_yaml_file, create_directories



In [23]:
# Update the configuration manager in src config

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):


        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [20]:
# Update the Components - This is the data ingestion components

import os
import urllib.request as request
import zipfile
import gdown
from Kidney_classifier import logger
from Kidney_classifier.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self) -> str:
        """
        Fetching data from the google drive
        """

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading dataset from {dataset_url} into the file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            full_download_url = prefix + file_id
            gdown.download(full_download_url, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into the file {zip_download_dir}")

        except Exception as e:
            raise e

    
    def extract_zip_file(self):
        """
        Args:
            zip_file_path: str
            Extracts the zip file into the data directory.

        Returns:
            None

        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [29]:
# Update the Pipeline - the state of execution of the workflow

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-08-01 11:40:51,039: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-01 11:40:51,042: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-01 11:40:51,046: INFO: common: created directory at: artifacts]
[2025-08-01 11:40:51,049: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-01 11:40:51,052: INFO: 3999868864: Downloading dataset from https://drive.google.com/file/d/10qNYUcIT9QIQ9m1yndnLk8NSNu83BrCk/view?usp=sharing into the file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=10qNYUcIT9QIQ9m1yndnLk8NSNu83BrCk
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=10qNYUcIT9QIQ9m1yndnLk8NSNu83BrCk&confirm=t&uuid=61a43310-5626-445e-a513-40cbdff2d52b
To: c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 41.9M/41.9M [00:03<00:00, 10.6MB/s]

[2025-08-01 11:40:57,292: INFO: 3999868864: Downloaded data from https://drive.google.com/file/d/10qNYUcIT9QIQ9m1yndnLk8NSNu83BrCk/view?usp=sharing into the file artifacts/data_ingestion/data.zip]
